In [1]:

import numpy as np
import pandas as pd
from sklearn import preprocessing
import lightgbm as lgb
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers import (Input, Dense, Concatenate)
from keras.utils import np_utils


Using TensorFlow backend.


In [2]:

def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:50].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    #plt.savefig('lgbm_importances01.png')



In [105]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
train_len = train_df.shape[0]
merged_df = pd.concat([train_df, test_df],sort=False)

In [106]:
var_list = [f for f in merged_df.columns if 'var' in f]
minmax_cols = [f+'_min_max' for f in var_list]

In [107]:

from scipy.special import erfinv

def rank_gauss(x):
    #Trying to implement rankGauss in python, here are my steps
    # 1) Get the index of the series
    # 2) sort the series
    # 3) standardize the series between -1 and 1
    # 4) apply erfinv to the standardized series
    # 5) create a new series using the index
    # Am i missing something ??
    # I subtract mean afterwards. And do not touch 1/0 (binary columns). 
    # The basic idea of this "RankGauss" was to apply rank trafo and them shape them like gaussians. 
    # Thats the basic idea. You can try your own variation of this.

    N = x.shape[0]
    temp = x.argsort()
    rank_x = temp.argsort() / N

    rank_x -= rank_x.mean()
    rank_x *= 2
    efi_x = erfinv(rank_x)
    efi_x -= efi_x.mean()
    
    return efi_x


def to_gauss(x): return np.sqrt(2)*erfinv(x) 

def normalize(data, exclude=None):
    norm_cols = data.columns
    n = data.shape[0]
    for col in norm_cols:
        sorted_idx = data[col].sort_values().index.tolist()
        uniform = np.linspace(start=-0.99, stop=0.99, num=n)
        normal = to_gauss(uniform)
        normalized_col = pd.Series(index=sorted_idx, data=normal)
        data.loc[:,col] = normalized_col.values
    return data

def DEEP_DAE(features_shape, act='relu'):
 
    # Input
    x = Input(name='inputs', shape=features_shape, dtype='float32')
    o = x
    
    # Encoder / Decoder
    o = Dense(1024, activation=act, name='dense1',kernel_initializer='random_normal', bias_initializer='zeros')(o)
    #o = Dense(1024, activation=act, name='dense2')(o)
    o = Dense(512, activation='relu', name='dense3')(o)
    #o = Dense(512, activation='relu', name='dense4')(o)
    o = Dense(256, activation='linear', name='bottle')(o)

    dec = Dense(200, activation='linear', name='dense_dec')(o) #200 original features
    
    # Print network summary
    Model(inputs=x, outputs=dec).summary()
    
    return Model(inputs=x, outputs=dec)




def FEATURES(model):
    input_ = model.get_layer('inputs').input
#     feat1 = model.get_layer('dense1').output
#     feat2 = model.get_layer('dense2').output
#     feat3 = model.get_layer('dense3').output
    feat_bottle = model.get_layer('bottle').output
    #feat = Concatenate(name='concat')([feat1, feat2, feat3])
    # use this if you want all layers
#     model = Model(inputs=[input_],
#                       outputs=[feat])

    model = Model(inputs=[input_],
                  outputs=[feat_bottle]) # only the last layer
    return model


In [108]:
# merged_df[var_list] = merged_df[var_list].transform(lambda x: rank_gauss(x))


# # merged_df[var_list] = normalize(merged_df[var_list])



In [109]:
np.random.seed(11)
noise_factor = 0.5
#merged_df_noisy = merged_df[var_list].add(noise_factor * np.random.normal(loc=0.0, scale=1.0, size=merged_df[var_list].shape))

merged_df_noisy = merged_df[var_list].add(noise_factor * np.random.normal(loc=0.0, scale=1.0, size=merged_df[var_list].shape))

In [112]:
# merged_df[minmax_cols] = normalize(merged_df[var_list])
# merged_df_noisy[minmax_cols] = normalize(merged_df_noisy[var_list])

merged_df[minmax_cols] = merged_df[var_list].transform(lambda x: rank_gauss(x))
merged_df_noisy[minmax_cols] = merged_df_noisy[var_list].transform(lambda x: rank_gauss(x))



In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

# merged_df[minmax_cols] = pd.DataFrame(scaler.fit_transform(merged_df[var_list]))
# merged_df_noisy[minmax_cols] = pd.DataFrame(scaler.fit_transform(merged_df_noisy[var_list]))

In [113]:
merged_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_0_min_max,var_1_min_max,var_2_min_max,var_3_min_max,var_4_min_max,var_5_min_max,var_6_min_max,var_7_min_max,var_8_min_max,var_9_min_max,var_10_min_max,var_11_min_max,var_12_min_max,var_13_min_max,var_14_min_max,var_15_min_max,var_16_min_max,var_17_min_max,var_18_min_max,var_19_min_max,var_20_min_max,var_21_min_max,var_22_min_max,var_23_min_max,var_24_min_max,var_25_min_max,var_26_min_max,var_27_min_max,var_28_min_max,var_29_min_max,var_30_min_max,var_31_min_max,var_32_min_max,var_33_min_max,var_34_min_max,var_35_min_max,var_36_min_max,var_37_min_max,var_38_min_max,var_39_min_max,var_40_min_max,var_41_min_max,var_42_min_max,var_43_min_max,var_44_min_max,var_45_min_max,var_46_min_max,var_47_min_max,var_48_min_max,var_49_min_max,var_50_min_max,var_51_min_max,var_52_min_max,var_53_min_max,var_54_min_max,var_55_min_max,var_56_min_max,var_57_min_max,var_58_min_max,var_59_min_max,var_60_min_max,var_61_min_max,var_62_min_max,var_63_min_max,var_64_min_max,var_65_min_max,var_66_min_max,var_67_min_max,var_68_min_max,var_69_min_max,var_70_min_max,var_71_min_max,var_72_min_max,var_73_min_max,var_74_min_max,var_75_min_max,var_76_min_max,var_77_min_max,var_78_min_max,var_79_min_max,var_80_min_max,var_81_min_max,var_82_min_max,var_83_min_max,var_84_min_max,var_85_min_max,var_86_min_max,var_87_min_max,var_88_min_max,var_89_min_max,var_90_min_max,var_91_min_max,var_92_min_max,var_93_min_max,var_94_min_max,var_95_min_max,var_96_min_max,var_97_min_max,var_98_min_max,var_99_min_max,var_100_min_max,var_101_min_max,var_102_min_max,var_103_min_max,var_104_min_max,var_105_min_max,var_106_min_max,var_107_min_max,var_108_min_max,var_109_min_max,var_110_min_max,var_111_min_max,var_112_min_max,var_113_min_max,var_114_min_max,var_115_min_max,var_116_min_max,var_117_min_max,var_118_min_max,var_119_min_max,var_120_min_max,var_121_min_max,var_122_min_max,var_123_min_max,var_124_min_max,var_125_min_max,var_126_min_max,var_127_min_max,var_128_min_max,var_129_min_max,var_130_min_max,var_131_min_max,var_132_min_max,var_133_min_max,var_134_min_max,var_135_min_max,var_136_min_max,var_137_min_max,var_138_min_max,var_139_min_max,var_140_min_max,var_141_min_max,var_142_min_max,var_143_min_max,var_144_min_max,var_145_min_max,var_146_min_max,var_147_min_max,var_148_min_max,var_149_min_max,var_150_min_max,var_151_min_max,var_152_min_max,var_153_min_max,var_154_min_max,var_155_min_max,var_156_min_max,var_157_min_max,var_158_min_max,var_159_min_max,var_160_min_max,var_161_min_max,var_162_min_

In [114]:
merged_df_noisy.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_0_min_max,var_1_min_max,var_2_min_max,var_3_min_max,var_4_min_max,var_5_min_max,var_6_min_max,var_7_min_max,var_8_min_max,var_9_min_max,var_10_min_max,var_11_min_max,var_12_min_max,var_13_min_max,var_14_min_max,var_15_min_max,var_16_min_max,var_17_min_max,var_18_min_max,var_19_min_max,var_20_min_max,var_21_min_max,var_22_min_max,var_23_min_max,var_24_min_max,var_25_min_max,var_26_min_max,var_27_min_max,var_28_min_max,var_29_min_max,var_30_min_max,var_31_min_max,var_32_min_max,var_33_min_max,var_34_min_max,var_35_min_max,var_36_min_max,var_37_min_max,var_38_min_max,var_39_min_max,var_40_min_max,var_41_min_max,var_42_min_max,var_43_min_max,var_44_min_max,var_45_min_max,var_46_min_max,var_47_min_max,var_48_min_max,var_49_min_max,var_50_min_max,var_51_min_max,var_52_min_max,var_53_min_max,var_54_min_max,var_55_min_max,var_56_min_max,var_57_min_max,var_58_min_max,var_59_min_max,var_60_min_max,var_61_min_max,var_62_min_max,var_63_min_max,var_64_min_max,var_65_min_max,var_66_min_max,var_67_min_max,var_68_min_max,var_69_min_max,var_70_min_max,var_71_min_max,var_72_min_max,var_73_min_max,var_74_min_max,var_75_min_max,var_76_min_max,var_77_min_max,var_78_min_max,var_79_min_max,var_80_min_max,var_81_min_max,var_82_min_max,var_83_min_max,var_84_min_max,var_85_min_max,var_86_min_max,var_87_min_max,var_88_min_max,var_89_min_max,var_90_min_max,var_91_min_max,var_92_min_max,var_93_min_max,var_94_min_max,var_95_min_max,var_96_min_max,var_97_min_max,var_98_min_max,var_99_min_max,var_100_min_max,var_101_min_max,var_102_min_max,var_103_min_max,var_104_min_max,var_105_min_max,var_106_min_max,var_107_min_max,var_108_min_max,var_109_min_max,var_110_min_max,var_111_min_max,var_112_min_max,var_113_min_max,var_114_min_max,var_115_min_max,var_116_min_max,var_117_min_max,var_118_min_max,var_119_min_max,var_120_min_max,var_121_min_max,var_122_min_max,var_123_min_max,var_124_min_max,var_125_min_max,var_126_min_max,var_127_min_max,var_128_min_max,var_129_min_max,var_130_min_max,var_131_min_max,var_132_min_max,var_133_min_max,var_134_min_max,var_135_min_max,var_136_min_max,var_137_min_max,var_138_min_max,var_139_min_max,var_140_min_max,var_141_min_max,var_142_min_max,var_143_min_max,var_144_min_max,var_145_min_max,var_146_min_max,var_147_min_max,var_148_min_max,var_149_min_max,var_150_min_max,var_151_min_max,var_152_min_max,var_153_min_max,var_154_min_max,var_155_min_max,var_156_min_max,var_157_min_max,var_158_min_max,var_159_min_max,var_160_min_max,var_161_min_max,var_162_min_max,var_163_min

In [117]:

batch_size = 128
epochs = 20
input_shape =  (merged_df_noisy[minmax_cols].shape[1], )

autoenc = DEEP_DAE(input_shape)
autoenc.compile(optimizer='adadelta', loss='mean_squared_error')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 200)               0         
_________________________________________________________________
dense1 (Dense)               (None, 1024)              205824    
_________________________________________________________________
dense3 (Dense)               (None, 512)               524800    
_________________________________________________________________
bottle (Dense)               (None, 256)               131328    
_________________________________________________________________
dense_dec (Dense)            (None, 200)               51400     
Total params: 913,352
Trainable params: 913,352
Non-trainable params: 0
_________________________________________________________________


In [118]:

autoenc.fit(merged_df_noisy[minmax_cols], merged_df[minmax_cols], epochs=epochs, 
            batch_size=batch_size, shuffle=True)




Epoch 1/20
400000/400000 [==============================] - 16s 39us/step - loss: 0.1527
Epoch 2/20
400000/400000 [==============================] - 15s 38us/step - loss: 0.0905
Epoch 3/20
400000/400000 [==============================] - 15s 38us/step - loss: 0.0817
Epoch 4/20
400000/400000 [==============================] - 15s 37us/step - loss: 0.0771
Epoch 5/20
400000/400000 [==============================] - 15s 38us/step - loss: 0.0743
Epoch 6/20
400000/400000 [==============================] - 15s 37us/step - loss: 0.0723
Epoch 7/20
400000/400000 [==============================] - 15s 37us/step - loss: 0.0708
Epoch 8/20
400000/400000 [==============================] - 15s 37us/step - loss: 0.0695
Epoch 9/20
400000/400000 [==============================] - 15s 38us/step - loss: 0.0685
Epoch 10/20
400000/400000 [==============================] - 15s 37us/step - loss: 0.0677
Epoch 11/20
400000/400000 [==============================] - 15s 38us/step - loss: 0.0670
Epoch 12/20
400000/

In [119]:
 
_model = FEATURES(autoenc)
features_encoded = _model.predict(merged_df_noisy[minmax_cols])
print(features_encoded.shape)


(400000, 256)


In [120]:
encoded_cols = ['encoded_var'+str(i) for i in range(features_encoded.shape[1])]

merged_df[encoded_cols] = pd.DataFrame(features_encoded)


In [121]:
merged_df.shape

(400000, 658)

In [123]:
merged_df.tail()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_0_min_max,var_1_min_max,var_2_min_max,var_3_min_max,var_4_min_max,var_5_min_max,var_6_min_max,var_7_min_max,var_8_min_max,var_9_min_max,var_10_min_max,var_11_min_max,var_12_min_max,var_13_min_max,var_14_min_max,var_15_min_max,var_16_min_max,var_17_min_max,var_18_min_max,var_19_min_max,var_20_min_max,var_21_min_max,var_22_min_max,var_23_min_max,var_24_min_max,var_25_min_max,var_26_min_max,var_27_min_max,var_28_min_max,var_29_min_max,var_30_min_max,var_31_min_max,var_32_min_max,var_33_min_max,var_34_min_max,var_35_min_max,var_36_min_max,var_37_min_max,var_38_min_max,var_39_min_max,var_40_min_max,var_41_min_max,var_42_min_max,var_43_min_max,var_44_min_max,var_45_min_max,var_46_min_max,var_47_min_max,...,encoded_var6,encoded_var7,encoded_var8,encoded_var9,encoded_var10,encoded_var11,encoded_var12,encoded_var13,encoded_var14,encoded_var15,encoded_var16,encoded_var17,encoded_var18,encoded_var19,encoded_var20,encoded_var21,encoded_var22,encoded_var23,encoded_var24,encoded_var25,encoded_var26,encoded_var27,encoded_var28,encoded_var29,encoded_var30,encoded_var31,encoded_var32,encoded_var33,encoded_var34,encoded_var35,encoded_var36,encoded_var37,encoded_var38,encoded_var39,encoded_var40,encoded_var41,encoded_var42,encoded_var43,encoded_var44,encoded_var45,encoded_var46,encoded_var47,encoded_var48,encoded_var49,encoded_var50,encoded_var51,encoded_var52,encoded_var53,encoded_var54,encoded_var55,encoded_var56,encoded_var57,encoded_var58,encoded_var59,encoded_var60,encoded_var61,encoded_var62,encoded_var63,encoded_var64,encoded_var65,encoded_var66,encoded_var67,encoded_var68,encoded_var69,encoded_var70,encoded_var71,encoded_var72,encoded_var73,encoded_var74,encoded_var75,encoded_var76,encoded_var77,encoded_var78,encoded_var79,encoded_var80,encoded_var81,encoded_var82,encoded_var83,encoded_var84,encoded_var85,encoded_var86,encoded_var87,encoded_var88,encoded_var89,encoded_var90,encoded_var91,encoded_var92,encoded_var93,encoded_var94,encoded_var95,encoded_var96,encoded_var97,encoded_var98,encoded_var99,encoded_var100,encoded_var101,encoded_var102,encoded_var103,encoded_var104,encoded_var105,encoded_var106,encoded_var107,encoded_var108,encoded_var109,encoded_var110,encoded_var111,encoded_var112,encoded_var113,encoded_var114,encoded_var115,encoded_var116,encoded_var117,encoded_var118,encoded_var119,encoded_var120,encoded_var121,encoded_var122,encoded_var123,encoded_var124,encoded_var125,encoded_var126,encoded_var127,encoded_var128,encoded_var129,encoded_var130,enc

In [124]:
train_df = merged_df.iloc[:train_len]
test_df = merged_df.iloc[train_len:]

In [125]:

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

n_folds = 5
random_seed = 26
model = '1024_auto_encoded'


model_name = "{0}_{1}_folds".format(model, n_folds)
print("Model: {}".format(model_name))

Model: 1024_auto_encoded_5_folds


In [126]:
exclusion = ['ID_code', 'target']  + minmax_cols + var_list

feats = [c for c in train_df.columns if c not in exclusion]
feats

['encoded_var0',
 'encoded_var1',
 'encoded_var2',
 'encoded_var3',
 'encoded_var4',
 'encoded_var5',
 'encoded_var6',
 'encoded_var7',
 'encoded_var8',
 'encoded_var9',
 'encoded_var10',
 'encoded_var11',
 'encoded_var12',
 'encoded_var13',
 'encoded_var14',
 'encoded_var15',
 'encoded_var16',
 'encoded_var17',
 'encoded_var18',
 'encoded_var19',
 'encoded_var20',
 'encoded_var21',
 'encoded_var22',
 'encoded_var23',
 'encoded_var24',
 'encoded_var25',
 'encoded_var26',
 'encoded_var27',
 'encoded_var28',
 'encoded_var29',
 'encoded_var30',
 'encoded_var31',
 'encoded_var32',
 'encoded_var33',
 'encoded_var34',
 'encoded_var35',
 'encoded_var36',
 'encoded_var37',
 'encoded_var38',
 'encoded_var39',
 'encoded_var40',
 'encoded_var41',
 'encoded_var42',
 'encoded_var43',
 'encoded_var44',
 'encoded_var45',
 'encoded_var46',
 'encoded_var47',
 'encoded_var48',
 'encoded_var49',
 'encoded_var50',
 'encoded_var51',
 'encoded_var52',
 'encoded_var53',
 'encoded_var54',
 'encoded_var55',
 '

In [ ]:

clfs = []
folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_seed)
oof_preds = np.zeros((len(train_df), 1))
test_preds = np.zeros((len(test_df), 1))


X = train_df[feats]
y = train_df['target']
X_test = test_df[feats]
test_ids = test_df.ID_code.values


parameters = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': 'true',
    #'scale_pos_weight': 400,
    #'device' : 'gpu' ,
    'boosting': 'gbdt',
    'num_leaves': 31, #31
    'feature_fraction': 0.5,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'learning_rate': 0.05, #0.05
    'verbose': 30
    #'min_data_in_leaf': 200
}

feature_importance_df = pd.DataFrame()
for fold_, (trn_, val_) in enumerate(folds.split(X, y)):
    print("Current Fold: {}".format(fold_+1))
    trn_x, trn_y = X.iloc[trn_, :], y[trn_]
    val_x, val_y = X.iloc[val_, :], y[val_]


    trn_lgb = lgb.Dataset(trn_x, trn_y)
    val_lgb = lgb.Dataset(val_x, val_y)
    clf = lgb.train(parameters,
                     train_set=trn_lgb,
                     #valid_sets=[valid_data_lgb,holdout_data_lgb],
                     valid_sets=[trn_lgb, val_lgb],
                     num_boost_round=3000,
                     early_stopping_rounds=50,
                     verbose_eval=False)
    


    val_pred = clf.predict(val_x)
    test_fold_pred = clf.predict(X_test)

    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    oof_preds[val_, :] = val_pred.reshape((-1, 1))
    test_preds += test_fold_pred.reshape((-1, 1))
    
   # print('getting feature importance')
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = feats
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    
test_preds /= n_folds
roc_score = roc_auc_score(y, oof_preds.ravel())
print("Overall AUC = {}".format(roc_score))


print("Saving submission file")
sample = pd.read_csv('../data/sample_submission.csv')
sample.target = test_preds.astype(float)
sample.ID_code = test_ids
sample.to_csv('../submissions/{}_{}.csv'.format(model_name,str(roc_score)), index=False)

display_importances(feature_importance_df)


Current Fold: 1
AUC = 0.8448114545252671
Current Fold: 2


In [77]:
get_ipython().run_cell_magic('javascript', '', 'var nb = IPython.notebook;\nvar kernel = IPython.notebook.kernel;\nvar command = "NOTEBOOK_FULL_PATH = \'" + nb.base_url + nb.notebook_path + "\'";\nkernel.execute(command);')


<IPython.core.display.Javascript object>

In [78]:
shutil.copyfile(os.path.basename(NOTEBOOK_FULL_PATH), 
                             '../models/{}_{}.ipynb'.format(model_name, str(roc_score)))


NameError: name 'NOTEBOOK_FULL_PATH' is not defined